https://www.youtube.com/watch?v=eBbEDRsCmv4&feature=youtu.be
https://www.tensorflow.org/get_started/summaries_and_tensorboard

In [65]:
import pandas as pd

In [35]:
def fibonacci_up_to(n):
    if n == 0:
        return [0]
    
    fibonaccis = [0, 1]
    k2 = 0
    k1 = 1
    for i in range(1, n):
        k0 = k1 + k2
        fibonaccis.append(k0)
        k2 = k1
        k1 = k0
    return fibonaccis

In [36]:
fibonacci_up_to(2), fibonacci_up_to(3), fibonacci_up_to(10)

([0, 1, 1], [0, 1, 1, 2], [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55])

In [13]:
f1000 = fibonacci_up_to(500)

In [14]:
f1000[-1]

86168291600238450732788312165664788095941068326060883324529903470149056115823592713458328176574447204501

In [144]:
x = []
y = []
N = 8
fib = fibonacci_up_to(2 + N + 1)
for i in range(2, 2 + N):
    x.append([fib[i], fib[i+1]])
    y.append(fib[i+2])
    
fib_data = pd.DataFrame(x)
fib_data.columns = ['x1', 'x2']
fib_data['y'] = y
fib_data = fib_data.sample(frac=1).reset_index(drop=True)
fib_data.head()

,x1,x2,y
0,34,55,89
1,3,5,8
2,13,21,34
3,8,13,21
4,2,3,5


## First let's create simple NN with two dimensional input that gives next element

In [145]:
import numpy as np
import tensorflow as tf

input_numbers = tf.placeholder(tf.float32, shape=[None, 2])

weight = tf.Variable(np.array([[1], [1]]), dtype=tf.float32, name="weight")
bias = tf.Variable([0], dtype=tf.float32)

output = tf.matmul(input_numbers, weight) + bias 
init = tf.global_variables_initializer()

In [146]:
feed_dict = {input_numbers: fib_data[['x1', 'x2']]}

In [147]:
with tf.Session() as sess:
    sess.run(init) 
    a = sess.run([output], feed_dict)
    a = np.sort(a, axis=1)
    print([int(k) for k in a[0][:,0]])

[3, 5, 8, 13, 21, 34, 55, 89]


## Let's train it :)

In [148]:
## Network configuration

input_numbers = tf.placeholder(tf.float32, shape=[None, 2], name="input")
output_numbers = tf.placeholder(tf.float32, shape=[None, 1], name="output")

weight = tf.Variable(tf.truncated_normal([2, 1]), dtype=tf.float32, name="weight")
bias = tf.Variable(tf.zeros([1]), dtype=tf.float32, name="bias")

prediction = tf.matmul(input_numbers, weight) + bias 
loss = tf.reduce_mean(tf.square(prediction - output_numbers))


In [165]:
## initilization 
epochs = 1000
learning_rate = 0.001

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

feed_dict = {input_numbers: fib_data[['x1', 'x2']], output_numbers: fib_data[['y']]}

In [170]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    #visualization
    writer = tf.summary.FileWriter("logs/1", sess.graph)
    
    for epoch in range(epochs):
        if epoch % 100 == 0:
            print('loss', sess.run(loss, feed_dict))
        sess.run(train, feed_dict)
    print(sess.run([weight, bias], feed_dict))
    a = sess.run(prediction, feed_dict)
    print(np.sort(a, axis=0))

loss 48.2583
loss 0.00725453
loss 0.00719081
loss 0.00713469
loss 0.00708488
loss 0.00704022
loss 0.00699979
loss 0.00696281
loss 0.00692879
loss 0.00689713
[array([[ 1.80179679],
       [ 0.50387168]], dtype=float32), array([ 0.0205179], dtype=float32)]
[[  2.8300581 ]
 [  5.13572645]
 [  7.94526625]
 [ 13.06047535]
 [ 20.98522377]
 [ 34.02518463]
 [ 54.98989105]
 [ 88.99454498]]
